# Hyperparameter tuning with Amazon SageMaker for molecular property prediction

*이 노트북은 [Hyperparameter tuning with Amazon SageMaker for molecular property prediction (영문 원본)](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/dgl_gcn_tox21/pytorch-gcn-tox21-hypertune.ipynb) 의 한국어 번역입니다.*

## Contents

1. [Background](##Background)  
2. [Setup](##Setup)  
3. [Code](##Code)  
4. [Tune](##Tune)  
5. [Wrap-up](##Wrap-up)  

## Background

이 예제 노트북은 자동 하이퍼파라미터 튜닝을 사용한 그래프 기반 분자 특성 예측 모델(graph-based molecular property prediction model)을 보여줍니다. 구현은 DGL 및 PyTorch를 기반으로 합니다. 최고의 하이퍼파라미터를 찾기 위해 SageMaker를 활용하여 다른 하이퍼파라미터 조합으로 여러 학습 작업들을 시작합니다. 이 예제에서는 [Amazon SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)를 사용하여 하이퍼파라미터 튜닝 작업을 생성합니다.

## Setup

이 노트북은 ml.p3.2xlarge 노트북 인스턴스에서 작성 및 테스트되었습니다.

선수 조건
 * 이 튜토리얼을 시작하기 전에 `pytorch-gcn-tox21.ipynb`을 검토하고 여러분 계정의 Amazon Elastic Container Registry(Amazon ECR)에
 \{account\}.dkr.ecr.\{region\}.amazonaws.com/sagemaker-dgl-pytorch-gcn-tox21:latest 가 있는지 확인하세요.
 * 학습 및 모델 데이터에 사용할 S3 버킷 및 접두사(prefix)를 확인하세요. S3는 노트북 인스턴스, 학습 및 호스팅과 동일한 리전 내에 있어야합니다.
 * 학습 및 호스팅 데이터 액세스를 제공하는 데 사용할 IAM 역할(role) ARN을 확인합니다. IAM 역할 생성에 대한 자세한 내용은 설명서를 참조하세요. 역할이 현재 노트북 인스턴스와 연결되어 있지 않거나 학습 및 호스팅에 둘 이상의 역할이 필요한 경우 `sagemaker.get_execution_role()`을 적절한 전체 IAM 역할 ARN 문자열로 바꿔야 합니다.

In [1]:
import sagemaker

from sagemaker import get_execution_role
from sagemaker.session import Session

# Setup session
sess = sagemaker.Session()

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = sess.default_bucket()

# Location to put your custom code.
custom_code_upload_location = 'customcode'

# IAM execution role that gives Amazon SageMaker access to resources in your AWS account.
# Use the Amazon SageMaker Python SDK to get the role from the notebook environment. 
role = get_execution_role()

## Code

Amazon SageMaker와 함께 Docker 컨테이너를 실행하려면 컨테이너를 실행할 Python 스크립트를 제공해야 합니다. 이 예제에서 `main.py`는 Amazon SageMaker 모델을 학습하는 데 필요한 모든 코드를 제공합니다.

In [2]:
!pygmentize main.py

import argparse
import dgl
import json
import numpy as np
import os
import random
import torch

from datetime import datetime
from dgl import model_zoo
from dgl.data.chem import Tox21
from dgl.data.utils import split_dataset
from sklearn.metrics import roc_auc_score
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

def setup(args, seed=0):
    args['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

    # Set random seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    return args

def collate_molgraphs(data):
    """Batching a list of datapoints for dataloader."""
    smiles, graphs, labels, masks = map(list, zip(*data))

    bg = dgl.batch(graphs)
    bg.set_n_initializer(dgl.init.zero_initializer)
    bg.set_e_initializer(dgl.init.zero_initializer)
    labels = torch.stack(labels, dim=0)
    masks = torch.stack(masks,

## Tune
Amazon SageMaker에서 단일 학습 작업을 학습하는 것과 유사하게 코드 스크립트, IAM 역할, (작업별) 하드웨어 설정 및 아직 튜닝하지 않은 하이퍼파라미터를 전달하는 학습 estimator를 정의합니다.

`pytorch-gcn-tox21.ipynb`에서 생성한 Amazon Elastic Container Registry(Amazon ECR)에 Docker 이미지가 있어야 합니다.

In [3]:
# Set target dgl-docker name
docker_name='sagemaker-dgl-pytorch-gcn-tox21'

CODE_PATH = 'main.py'
code_location = sess.upload_data(CODE_PATH, bucket=bucket, key_prefix=custom_code_upload_location)

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, docker_name)

estimator = sagemaker.estimator.Estimator(image,
                                          role, 
                                          train_instance_count=1, 
                                          train_instance_type='ml.p3.2xlarge',
                                          hyperparameters={'entrypoint': CODE_PATH},
                                          sagemaker_session=sess)

Estimator 객체를 정의한 후, 조정하려는 하이퍼파라미터와 가능한 값들을 지정하세요. 가능한 값들의 유형에 따라 하이퍼파라미터는 세 가지 클래스로 나눌 수 있습니다.

* **범주형(Categorical)**:: 가능한 범주값들로 이산(discrete) 셋을 구성하며 `CategoricalParameter(list)`로 표시됩니다.
* **연속형(Continuous)** : `[min, max]` 간격 내에서 임의의 실수를 사용할 수 있으며, `ContinuousParameter(min, max)`로 표시됩니다.
* **정수형(Integer)**: `[min, max]` 간격 내에서 정수 값을 사용할 수 있으며 `IntegerParameter (min, max)`로 표시됩니다.

값을 최소 제한 유형(least restrictive type)으로 지정하는 것이 대부분 좋습니다. 예를 들어 `ContinuousParameter(0.01, 0.2)`는 `CategoricalParameter([0.01, 0.1, 0.15, 0.2])`보다 덜 제한적입니다.

In [4]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter

hyper_ranges = {'lr': ContinuousParameter(1e-4, 1e-2),
                'patience': IntegerParameter(5, 30),
                'n_hidden': CategoricalParameter([32, 64, 128])}

다음으로 튜닝할 목표 지표(metric)와 지표에 대한 정의를 지정하십시오. 여기에는 학습 작업의 Amazon CloudWatch 로그에서 해당 지표를 추출하는 데 필요한 정규식(regex)이 포함됩니다.

In [5]:
objective_name = 'Validation_roc_auc'
metric_definitions = [{'Name': objective_name,
                       'Regex': 'Best validation score ([0-9\\.]+)'}]

이제 `HyperparameterTuner` 객체를 생성하겠습니다. 해당 객체의 인자값에는 아래와 같은 값들이 포함되어야 합니다.

* 위에서 생성한 학습 estimator
* 하이퍼파라미터 범위
* 목표 지표 이름 및 정의
* 총 학습 작업 수와 동시에 실행해야 하는 학습 작업 수; 병렬 작업이 많을수록 튜닝이 더 빨리 완료되지만 정확도(accuracy)가 떨어질 수 있습니다. 병렬 작업 값을 총 학습 작업 수의 10% 미만으로 설정하는 것이 좋습니다. 이 예제에서는 빠르게 실행하기 위해 동시 학습 작업 수가 높게 설정되어 있습니다.
* 목표 지표을 최대화해야하는지 또는 최소화해야하는지 여부(Maximize or Minimize); 기본값이 'Maximize'이고 이 예제에서는 roc-auc를 최대화하는 것이기 때문에 기본값을 지정하지 않습니다.

In [6]:
from sagemaker.tuner import HyperparameterTuner

tuner = HyperparameterTuner(estimator,
                            objective_name,
                            hyper_ranges,
                            metric_definitions,
                            max_jobs=6,
                            max_parallel_jobs=2)

마지막으로 `.fit()`을 호출하여 튜닝 작업을 시작하세요.

In [7]:
tuner.fit(inputs={'training-code': code_location})

하이퍼파라미터 튜닝 작업 상태를 점검하여 성공적으로 시작되었고 진행 중(InProgress)인지 확인하세요.

In [8]:
import boto3

boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'InProgress'

## Wrap-up
하이퍼파라미터 튜닝 작업이 시작된 후, 백그라운드에서 실행되며 이 노트북을 닫을 수 있습니다. 완료되면 콘솔로 이동하여 결과를 분석할 수 있습니다.

Amazon SageMaker의 하이퍼파라메터 튜닝에 대한 자세한 내용은 AWS 설명서를 참조하세요.